In [1]:
# system packages
import sys
import time
import warnings
import os

# non-geo numeric packages
import numpy as np
import math
from itertools import product, combinations
import pandas as pd

# network and OSM packages
import networkx as nx
import osmnx as ox
city_geo = ox.geocoder.geocode_to_gdf

# Earth engine packages
import ee
import geemap

# General geo-packages
import libpysal
import rasterio
import geopandas as gpd
import shapely
from shapely import geometry
from shapely.geometry import Point, MultiLineString, LineString, Polygon, MultiPolygon

In [2]:
# Authenticate and Initialize Google Earth Engine
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AVHEtk54M9lXpJZmq6XFNBSHtB9a5SsmpGNtVe-ZuHFqFzT-Wq5hRHF5p70

Successfully saved authorization token.


In [3]:
# Block 0 cities and thresholds
thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m

# Extract iso-3166 country codes
iso = pd.read_excel('iso_countries.xlsx')

# Extract cities list
cities = pd.read_excel('cities.xlsx') # all cities

# 'cities_adj' serves by default as city-input for functions
# cities_adj = cities
# cities_adj = cities[cities['Included (Y/N)'] == 'Y']
cities_adj = cities[cities['City'].isin(['Addis Ababa','Dhaka','Shijiazhuang','Damascus'])]
cities_adj = cities_adj.reset_index()

In [5]:
%%time
# 1. Required preprocess for information extraction
warnings.filterwarnings('ignore')

# In essence, we use Google Earth Engine to extract a country's grid raster and clip it with the city's preferred OSM area
# Predifine in Excel: the (1) city name as "City" and (2) the OSM area that needs to be extracted as "OSM_area"
# i.e. City = "Los Angeles" and OSM_area = "Los Angeles county, Orange county CA"
files = gee_worldpop_extract(cities_adj, iso, 'C:/Dumps/GEE_city_grids/')

# Files are downloaded automatically to the specified path. Files are also stored in Google with a downloadlink:

Generating URL ...
Please wait ...
Data downloaded to C:\Dumps\GEE_city_grids\ETH_Addis Ababa_2020.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Dumps\GEE_city_grids\SYR_Damascus_2020.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Dumps\GEE_city_grids\BGD_Dhaka_2020.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Dumps\GEE_city_grids\CHN_Shijiazhuang_2020.tif
Wall time: 13.3 s


In [37]:
%%time
# 2. Information extraction

# Clip cities from countries, format population grids
population_grids = city_grids_format(files,
                                     cities_adj['OSM_area'],
                                     grid_size = 100) # aggregating upwards to i.e. 200m, 300m etc. is possible
print(' ')

# Get road networks
road_networks = road_networks(cities_adj, # Get 'all' (drive,walk,bike) network
                              thresholds,
                              undirected = True)
print(' ')

# Extract urban greenspace (UGS)
UGS = urban_greenspace(cities_adj, 
                       thresholds,
                       one_UGS_buf = 25, # buffer at which UGS is seen as one
                       min_UGS_size = 400) # WHO sees this as minimum UGS size (400m2)

100m resolution grids extraction
Addis Ababa 0.4 mns
Damascus 0.53 mns
Dhaka 0.79 mns
Shijiazhuang 1.14 mns
 
get road networks from OSM
Addis Ababa done 1.65 mns
Damascus done 2.22 mns
Dhaka done 3.3 mns
Shijiazhuang done 3.67 mns
 
get urban greenspaces from OSM
Addis Ababa done
Damascus done
Dhaka done
Shijiazhuang done
Wall time: 4min 50s


In [38]:
%%time
# 3. Preprocess information for route finding

# Get fake entry points (between UGS and buffer limits)
UGS_entry = UGS_fake_entry(UGS, 
                           road_networks['nodes'], 
                           cities_adj['City'],
                           UGS_entry_buf = 25, # road nodes within 25 meters are seen as fake entry points
                           walk_radius = 500, # assume that the average person only views a UGS up to 500m in radius
                                                # more attractive
                           entry_point_merge = 0) # merges closeby fake UGS entry points within X meters 
                                                    # what may be done for performance
print(' ')
# Checks all potential suitible combinations (points that fall within max threshold Euclidean distance from the ego)
suitible = suitible_combinations(UGS_entry, 
                                 population_grids, 
                                 road_networks['nodes'], # For finding nearest grid entry points
                                 thresholds,
                                 cities_adj['City'],
                                 chunk_size = 10000000) # calculating per chunk of num UGS entry points * num pop_grids
                                                        # Preventing normal PC meltdown, set lower if PC gets stuck
print(' ')
# Checks if grids are already in a UGS
suitible_InOut_UGS = grids_in_UGS (suitible, UGS, population_grids)

get fake UGS entry points
Addis Ababa 0.0 % done 0.01  mns
Addis Ababa 73.5 % done 0.39  mns
Addis Ababa 100 % done 0.53  mns
Damascus 0.0 % done 0.53  mns
Damascus 41.2 % done 0.67  mns
Damascus 82.3 % done 0.81  mns
Damascus 100 % done 0.88  mns
Dhaka 0.0 % done 0.89  mns
Dhaka 19.4 % done 1.12  mns
Dhaka 38.8 % done 1.33  mns
Dhaka 58.1 % done 1.56  mns
Dhaka 77.5 % done 1.77  mns
Dhaka 96.9 % done 2.01  mns
Dhaka 100 % done 2.05  mns
Shijiazhuang 0.0 % done 2.05  mns
Shijiazhuang 100 % done 2.14  mns
 
get potential (Euclidean) suitible combinations
Addis Ababa
in chunk 1 / 7 87293 suitible comb.
in chunk 2 / 7 3449 suitible comb.
in chunk 3 / 7 9022 suitible comb.
in chunk 4 / 7 1081 suitible comb.
in chunk 5 / 7 6206 suitible comb.
in chunk 6 / 7 36604 suitible comb.
in chunk 7 / 7 53040 suitible comb.
total combinations within distance 196695
0.0 % gridentry done 0.0  mns
100 % gridentry done 2.48  mns
Damascus
in chunk 1 / 4 41477 suitible comb.
in chunk 2 / 4 83841 suitible co

In [39]:
%%time
# 4. Finding shortest routes.
Routes = route_finding (road_networks['graphs'], # graphs of the road networks
               suitible_InOut_UGS, # potential suitible routes with grid-UGS comb. separated in or out UGS.
               road_networks['nodes'], 
               road_networks['edges'], 
               cities_adj['City'], 
               block_size = 250000, # Chunk to spread dataload.
               nn_iter = 10) # max amount of nearest nodes to be found (both for UGS entry and grid-centroid road entries)


Addis Ababa 1 / 1 range 0 - 169540
0.0 % done 0.0 mns
5.9 % done 0.1 mns
11.8 % done 0.23 mns
17.69 % done 0.39 mns
23.59 % done 0.54 mns
29.49 % done 0.66 mns
35.39 % done 0.81 mns
41.29 % done 1.03 mns
47.19 % done 1.2 mns
53.08 % done 1.35 mns
index 96883 No route
index 96884 No route
index 96885 No route
index 96886 No route
index 96982 No route
index 96983 No route
index 96985 No route
index 96986 No route
index 96987 No route
58.98 % done 1.95 mns
64.88 % done 2.03 mns
70.78 % done 2.12 mns
76.68 % done 2.21 mns
82.58 % done 2.31 mns
88.47 % done 2.39 mns
94.37 % done 2.52 mns
for 32 routes nearest nodes found
100.0 % pathfinding done 2.61 mns
formatting done 3.53 mns
dissolving done 4.51 mns
Damascus 1 / 2 range 0 - 250000
0.0 % done 4.52 mns
2.48 % done 9.2 mns
4.97 % done 9.48 mns
7.45 % done 10.13 mns
9.93 % done 10.25 mns
12.42 % done 10.35 mns
14.9 % done 10.51 mns
17.38 % done 10.69 mns
19.87 % done 10.87 mns
22.35 % done 11.1 mns
24.83 % done 11.23 mns
27.32 % done 11.41 

index 402615 No route
index 402616 No route
index 402617 No route
index 402618 No route
index 402619 No route
index 402620 No route
index 402621 No route
index 402636 No route
index 402637 No route
index 402638 No route
index 402639 No route
index 402640 No route
index 402641 No route
index 402642 No route
index 402643 No route
index 402644 No route
index 402645 No route
index 402646 No route
index 402647 No route
index 402648 No route
index 402649 No route
index 402650 No route
index 402651 No route
index 402652 No route
index 402653 No route
index 402654 No route
index 402655 No route
index 402656 No route
index 402657 No route
index 402658 No route
index 402659 No route
index 402660 No route
index 402661 No route
index 402662 No route
index 402663 No route
index 402664 No route
index 402665 No route
index 402666 No route
index 402667 No route
index 402668 No route
index 402669 No route
index 402670 No route
index 402671 No route
index 402672 No route
index 402673 No route
index 4026

In [48]:
%%time
# 5. summarize scores
min_gridUGS = min_gridUGS_comb (Routes, population_grids, UGS)

E2SCFA_score = E2SCFA_scores(min_gridUGS, 
                             population_grids, 
                             thresholds, 
                             cities_adj['City'], 
                             save_path = 'C:/Dumps/GEE-WP Scores/E2SFCA/', 
                             grid_size = 100)

E2SCFA_score['score summary']

300 Addis Ababa
600 Addis Ababa
1000 Addis Ababa
300 Damascus
600 Damascus
1000 Damascus
300 Dhaka
600 Dhaka
1000 Dhaka
300 Shijiazhuang
600 Shijiazhuang
1000 Shijiazhuang
Wall time: 56.3 s


City,Addis Ababa,Damascus,Dhaka,Shijiazhuang
population,"3,552,822.00","2,075,713.00","12,219,271.00","2,979,832.00"
Sc-access 300,72.47,1.30,3.79,38.17
M-dist 300,6.83,40.38,15.92,5.09
M-area 300,"8,329,597.37","14,559.79","35,965.75","29,968.11"
M-supply 300,74.40,1.05,4.16,41.60
Sc-norm 300,3.74,0.01,0.23,9.15
Sc-access 600,64.06,1.30,3.13,36.90
M-dist 600,30.58,128.85,72.75,19.87
M-area 600,"9,501,572.99","17,957.38","55,841.92","46,858.40"
M-supply 600,66.01,0.87,3.28,41.00


In [4]:
def gee_worldpop_extract (city_file, iso, save_path = None):
    
    cities = city_file
    
    # Get included city areas
    OSM_incl = [cities[cities['City'] == city]['OSM_area'].tolist()[0].rsplit(', ') for city in cities['City'].tolist()]

    # Get the city geoms
    obj = [city_geo(city).dissolve()['geometry'].tolist()[0] for city in OSM_incl]

    # Get the city countries
    obj_displ = [city_geo(city).dissolve()['display_name'].tolist()[0].rsplit(', ')[-1]for city in OSM_incl]
    obj_displ = np.where(pd.Series(obj_displ).str.contains("Ivoire"),"CIte dIvoire",obj_displ)

    # Get the country's iso-code
    iso_list = [iso[iso['name'] == ob]['alpha3'].tolist()[0] for ob in obj_displ]

    # Based on the iso-code return the worldpop 2020
    ee_worldpop = [ee.ImageCollection("WorldPop/GP/100m/pop")\
        .filter(ee.Filter.date('2020'))\
        .filter(ee.Filter.inList('country', [io])).first() for io in iso_list]

    # Clip the countries with the city geoms.
    clipped = [ee_worldpop[i].clip(shapely.geometry.mapping(obj[i])) for i in range(0,len(obj))]

    # Create path if non-existent
    if save_path == None:
        path = ''
    else:
        path = save_path
        if not os.path.exists(path):
                    os.makedirs(path)

    # Export as TIFF file.
    # Stored in form path + USA_Los Angeles_2020.tif
    filenames = [path+iso_list[i]+'_'+cities['City'][i]+'_2020.tif' for i in range(len(obj))]
    [geemap.ee_export_image(clipped[i], filename = filenames[i]) for i in range(0,len(obj))]
    return(filenames)
    sys.stdout.flush()

In [21]:
# Block 2 population grids extraction
def city_grids_format(city_grids, cities_area, grid_size = 100):
    start_time = time.time()
    grids = []
    print(str(grid_size) + 'm resolution grids extraction')
    for i in range(len(city_grids)):
        
        # Open the raster file
        with rasterio.open(city_grids[i]) as src:
            band= src.read() # the population values
            aff = src.transform # the raster bounds and size (affine)
        
        # Get the rowwise arrays, get a 2D dataframe
        grid = pd.DataFrame()
        for b in enumerate(band[0]):
            grid = pd.concat([grid, pd.Series(b[1],name=b[0])],axis=1)
        grid= grid.unstack().reset_index()
        
        # Unstack df to columns
        grid.columns = ['row','col','value']
        grid['minx'] = aff[2]+aff[0]*grid['col']
        grid['miny'] = aff[5]+aff[4]*grid['row']
        grid['maxx'] = aff[2]+aff[0]*grid['col']+aff[0]
        grid['maxy'] = aff[5]+aff[4]*grid['row']+aff[4]
        
        # Create polygon from affine bounds and row/col indices
        grid['geometry'] = [Polygon([(grid.minx[i],grid.miny[i]),
                                   (grid.maxx[i],grid.miny[i]),
                                   (grid.maxx[i],grid.maxy[i]),
                                   (grid.minx[i],grid.maxy[i])])\
                          for i in range(len(grid))]
        
        # Set the df as geo-df
        grid = gpd.GeoDataFrame(grid, crs = 4326) 

        # Get dissolvement_key for dissolvement. 
        grid['row3'] = np.floor(grid['row']/(grid_size/100)).astype(int)
        grid['col3'] = np.floor(grid['col']/(grid_size/100)).astype(int)
        grid['dissolve_key'] = grid['row3'].astype(str) +'-'+ grid['col3'].astype(str)
        
        # Define a city's OSM area as Polygon.
        geo_ls = gpd.GeoSeries(city_geo(cities_area[i].split(', ')).dissolve().geometry)
        
        # Intersect grids with the city boundary Polygon.
        insec = grid.intersection(geo_ls.tolist()[0])
        
        # Exclude grids outside the specified city boundaries
        insec = insec[insec.area > 0]
        
        # Join in other information.
        insec = gpd.GeoDataFrame(geometry = insec, crs = 4326).join(grid.loc[:, grid.columns != 'geometry'])
        
        # Dissolve into block by block grids
        popgrid = insec[['dissolve_key','geometry','row3','col3']].dissolve('dissolve_key')
        
        # Get those grids populations and area. Only blocks with population and full blocks
        popgrid['population'] = round(insec.groupby('dissolve_key')['value'].sum()).astype(int)
        popgrid['area_m'] = round(gpd.GeoSeries(popgrid['geometry'], crs = 4326).to_crs(3043).area).astype(int)
        popgrid = popgrid[popgrid['population'] > 0]
        popgrid = popgrid[popgrid['area_m'] / popgrid['area_m'].max() > 0.95]

        # Get centroids and coords
        popgrid['centroid'] = popgrid['geometry'].centroid
        popgrid['centroid_m'] = gpd.GeoSeries(popgrid['centroid'], crs = 4326).to_crs(3043)
        popgrid['grid_lon'] = popgrid['centroid_m'].x
        popgrid['grid_lat'] = popgrid['centroid_m'].y
        popgrid = popgrid.reset_index()

        minx = popgrid.bounds['minx']
        maxx = popgrid.bounds['maxx']
        miny = popgrid.bounds['miny']
        maxy = popgrid.bounds['maxy']

        # Some geometries result in a multipolygon when dissolving (like i.e. 0.05 meters), coords error.
        # Therefore recreate the polygon.
        Poly = []
        for k in range(len(popgrid)):
            Poly.append(Polygon([(minx[k],maxy[k]),(maxx[k],maxy[k]),(maxx[k],miny[k]),(minx[k],miny[k])]))
        popgrid['geometry'] = Poly

        grids.append(popgrid)

        print(city_grids[i].rsplit('_')[3], round((time.time() - start_time)/60,2),'mns')
    return(grids)

In [22]:
# Block 3 Road networks
def road_networks (cities, thresholds, undirected = False):
    print('get road networks from OSM')
    start_time = time.time()
    graphs = list()
    road_nodes = list()
    road_edges = list()
    road_conn = list()

    for i in enumerate(cities['OSM_area']):
        # Get graph, road nodes and edges
        road_node = pd.DataFrame()
        roads = pd.DataFrame()
        
        # For each included OSM_area get the roads
        for district in i[1].rsplit(', '):
            graph = ox.graph_from_place(district, network_type = "all", buffer_dist = (np.max(thresholds)+1000))
            node, edge = ox.graph_to_gdfs(graph)
            road_node = pd.concat([road_node, node], axis = 0)
            roads = pd.concat([roads, edge], axis = 0)
        
        # Eliminate lists in the df which prevents drop of duplicate columns
        road_edge = pd.DataFrame([[c[0] if isinstance(c,list) else c for c in roads[col]]\
                              for col in roads]).transpose()
        road_edge.columns = roads.columns
        road_edge.index = roads.index
        road_edge = gpd.GeoDataFrame(road_edge, crs = 4326)
        
        # Return the unique nodes and edges of the (often) adjacent OSM_areas.
        road_node = road_node.drop_duplicates()
        road_edge = road_edge.drop_duplicates()
        
        # Road nodes format
        road_node = road_node.to_crs(4326)
        road_node['geometry_m'] = gpd.GeoSeries(road_node['geometry'], crs = 4326).to_crs(3043)
        road_node['osmid_var'] = road_node.index
        road_node = gpd.GeoDataFrame(road_node, geometry = 'geometry', crs = 4326)

        # format road edges
        road_edge['geometry_m'] = gpd.GeoSeries(road_edge['geometry'], crs = 4326).to_crs(3043)
        road_edge = road_edge.reset_index()
        road_edge.rename(columns={'u':'from', 'v':'to', 'key':'keys'}, inplace=True)
        road_edge['key'] = road_edge['from'].astype(str) + '-' + road_edge['to'].astype(str)
        
        if undirected == True:
            # Apply one-directional to both for walking
            both = road_edge[road_edge['oneway'] == False]
            one = road_edge[road_edge['oneway'] == True]
            rev = pd.DataFrame()
            rev[['from','to']] = one[['to','from']]
            rev = pd.concat([rev,one.iloc[:,2:]],axis = 1)
            edge_bidir = pd.concat([both, one, rev])
            edge_bidir = edge_bidir.reset_index()
            edge_bidir['oneway'] = False
        else:
            edge_bidir = road_edge

        # Exclude highways and ramps on edges    
        edge_filter = edge_bidir[(edge_bidir['highway'].str.contains('motorway') | 
              (edge_bidir['highway'].str.contains('trunk') & 
               edge_bidir['maxspeed'].astype(str).str.contains(
                   '40 mph|45 mph|50 mph|55 mph|60 mph|65|70|75|80|85|90|95|100|110|120|130|140'))) == False]
        road_edges.append(edge_filter)

        # Exclude isolated nodes
        fltrnodes = pd.Series(list(edge_filter['from']) + list(edge_filter['to'])).unique()
        newnodes = road_node[road_node['osmid_var'].isin(fltrnodes)]
        road_nodes.append(newnodes)

        # Get only necessary road connections columns for network performance
        road_con = edge_filter[['osmid','key','length','geometry']]
        road_con = road_con.set_index('key')

        road_conn.append(road_con)

        # formatting to graph again.
        newnodes = newnodes.loc[:, ~newnodes.columns.isin(['geometry_m', 'osmid_var'])]
        edge_filter = edge_filter.set_index(['from','to','keys'])
        edge_filter = edge_filter.loc[:, ~edge_filter.columns.isin(['geometry_m', 'key'])]

        graph2 = ox.graph_from_gdfs(newnodes, edge_filter)

        graphs.append(graph2)
        print(cities['City'][i[0]].rsplit(',')[0], 'done', round((time.time() - start_time) / 60,2),'mns')
    return({'graphs':graphs,'nodes':road_nodes,'edges':road_conn,'edges long':road_edges})

In [23]:
# Block 4 city greenspace
def urban_greenspace (cities, thresholds, one_UGS_buf = 25, min_UGS_size = 400):
    print('get urban greenspaces from OSM')
    parks_in_range = list()
    for i in enumerate(cities['OSM_area']):
        # Tags seen as Urban Greenspace (UGS) require the following:
        # 1. Tag represent an area
        # 2. The area is outdoor
        # 3. The area is (semi-)publically available
        # 4. The area is likely to contain trees, grass and/or greenery
        # 5. The area can reasonable be used for walking or recreational activities
        tags = {'landuse':['allotments','forest','greenfield','village_green'],\
                'leisure':['garden','fitness_station','nature_reserve','park','playground'],\
                'natural':'grassland'}
        gdf = ox.geometries_from_place(i[1].rsplit(', '),tags = tags,buffer_dist = np.max(thresholds))
        gdf = gdf[(gdf.geom_type == 'Polygon') | (gdf.geom_type == 'MultiPolygon')]
        greenspace = gdf.reset_index()    
        warnings.filterwarnings("ignore")

        green_buffer = gpd.GeoDataFrame(geometry = greenspace.to_crs(3043).buffer(one_UGS_buf).to_crs(4326))
        greenspace['geometry_w_buffer'] = green_buffer
        greenspace['geometry_w_buffer'] = gpd.GeoSeries(greenspace['geometry_w_buffer'], crs = 4326)
        greenspace['geom buffer diff'] = greenspace['geometry_w_buffer'].difference(greenspace['geometry'])

        # This function group components in itself that overlap (with the buffer set of 25 metres)
        # https://stackoverflow.com/questions/68036051/geopandas-self-intersection-grouping
        W = libpysal.weights.fuzzy_contiguity(greenspace['geometry_w_buffer'])
        greenspace['components'] = W.component_labels
        parks = greenspace.dissolve('components')

        # Exclude parks below 0.04 ha.
        parks = parks[parks.to_crs(3043).area > min_UGS_size]
        print(cities['City'][i[0]], 'done')
        parks = parks.reset_index()
        parks['geometry_m'] = parks['geometry'].to_crs(3043)
        parks['park_area'] = parks['geometry_m'].area
        parks_in_range.append(parks)
    return(parks_in_range)

In [24]:
# Block 5 park entry points
def UGS_fake_entry(UGS, road_nodes, cities, UGS_entry_buf = 25, walk_radius = 500, entry_point_merge = 0):
    print('get fake UGS entry points')
    start_time = time.time()
    ParkRoads = list()
    for j in range(len(cities)):
        ParkRoad = pd.DataFrame()
        mat = list()
        # For all
        for i in range(len(UGS[j])):
            dist = road_nodes[j]['geometry'].to_crs(3043).distance(UGS[j]['geometry'].to_crs(
                3043)[i])
            buf_nodes = road_nodes[j][(dist < UGS_entry_buf) & (dist > 0)]
            mat.append(list(np.repeat(i, len(buf_nodes))))
            ParkRoad = pd.concat([ParkRoad, buf_nodes])
            if i % 100 == 0: print(cities[j].rsplit(',')[0], round(i/len(UGS[j])*100,1),'% done', 
                                  round((time.time() - start_time) / 60,2),' mns')
        # Park no list conversion
        mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, mat) for i in b]

        # Format
        ParkRoad['Park_No'] = mat_u
        ParkRoad = ParkRoad.reset_index()
        ParkRoad['park_lon'] = ParkRoad['geometry_m'].x
        ParkRoad['park_lat'] = ParkRoad['geometry_m'].y
        
        # Get the road nodes intersecting with the parks' buffer
        ParkRoad = pd.merge(ParkRoad, UGS[j][['geometry','park_area']], left_on = 'Park_No', right_index = True)

        # Get the walkable park size
        ParkRoad['park_size_walkable'] = ParkRoad['geometry_m'].buffer(walk_radius).to_crs(4326).intersection(ParkRoad['geometry_y'].to_crs(4326))
        ParkRoad['walk_area'] = ParkRoad['park_size_walkable'].to_crs(3043).area
        #ParkRoad['park_area'] = ParkRoad['geometry_y'].to_crs(3043).area
        ParkRoad['share_walked'] = ParkRoad['walk_area'] / ParkRoad['park_area']
                
        # Merge fake UGS entry points if within X meters of each other for better system performance
        # Standard no merging
        ParkRoad = simplify_UGS_entry(ParkRoad, entry_point_merge = 0)
                
        ParkRoads.append(ParkRoad)

        print(cities[j].rsplit(',')[0],'100 % done', 
                                  round((time.time() - start_time) / 60,2),' mns')
    return(ParkRoads)

In [25]:
# Block 5.5 (not in use, buffer is 0, thus retains all the park entry points as is)
def simplify_UGS_entry(fake_UGS_entry, entry_point_merge = 0):
    # Get buffer of nodes close to each other.
    # Get the buffer
    ParkComb = fake_UGS_entry
    ParkComb['geometry_m_buffer'] = ParkComb['geometry_m'].buffer(entry_point_merge)

    # Get and merge components
    M = libpysal.weights.fuzzy_contiguity(ParkComb['geometry_m_buffer'])
    ParkComb['components'] = M.component_labels

    # Take centroid of merged components
    centr = gpd.GeoDataFrame(ParkComb, geometry = 'geometry_x', crs = 4326).dissolve('components')['geometry_x'].centroid
    centr = gpd.GeoDataFrame(centr)
    centr.columns = ['comp_centroid']

    # Get node closest to the centroid of all merged nodes, which accesses the road network.
    ParkComb = pd.merge(ParkComb, centr, left_on = 'components', right_index = True)
    ParkComb['centr_dist'] = ParkComb['geometry_x'].distance(ParkComb['comp_centroid'])
    ParkComb = ParkComb.iloc[ParkComb.groupby('components')['centr_dist'].idxmin()]
    return(ParkComb)

In [26]:
# Block 6 grid-parkentry combinations within euclidean threshold distance
def suitible_combinations(UGS_entry, pop_grids, road_nodes, thresholds, cities, chunk_size = 10000000):
    print('get potential (Euclidean) suitible combinations')
    start_time = time.time()
    RoadComb = list()
    for l in range(len(cities)):
        #blockA = block_combinations
        print(cities[l])
        len1 = len(pop_grids[l])
        len2 = len(UGS_entry[l])

        # Reduce the size of combinations per iteration
        len4 = 1
        len5 = len1 * len2
        blockC = len5
        while blockC > chunk_size:
            blockC = len5 / len4
            #print(blockC, len4)
            len4 = len4+1

        # Amount of grids taken per iteration block
        block = round(len1 / len4)

        output = pd.DataFrame()
        # Checking all the combinations at once is too performance intensive, it is broken down per 1000 (or what you want)
        for i in range(len4):
            # Check all grid-park combinations per block
            l1, l2 = range(i*block,(i+1)*block), range(0,len2)
            listed = pd.DataFrame(list(product(l1, l2)))

            # Merge grid and park information
            grid_merged = pd.merge(listed, 
                                   pop_grids[l][['grid_lon','grid_lat','centroid','centroid_m']],
                                   left_on = 0, right_index = True)
            node_merged = pd.merge(grid_merged, 
                                   UGS_entry[l][['Park_No','osmid','geometry_x','geometry_y','geometry_m','park_lon','park_lat',
                                       'share_walked','park_area','walk_area']], 
                                   left_on = 1, right_index = True)

            # Preset index for merging
            node_merged['key'] = range(0,len(node_merged))
            node_merged = node_merged.set_index('key')
            node_merged = node_merged.loc[:, ~node_merged.columns.isin(['index'])]

            # Create lists for better computational performance
            glon = list(node_merged['grid_lon'])
            glat = list(node_merged['grid_lat'])
            plon = list(node_merged['park_lon'])
            plat = list(node_merged['park_lat'])

            # Get the euclidean distances
            mat = list()
            for j in range(len(node_merged)):
                mat.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2))

            # Check if distances are within 1000m and join remaining info and concat in master df per 1000.
            mat_df = pd.DataFrame(mat)[(np.array(mat) <= np.max(thresholds))]

            # join the other gravity euclidean scores and other information
            mat_df.columns = ['Euclidean']    
            mat_df = mat_df.join(node_merged)

            output = pd.concat([output, mat_df])

            print('in chunk',(i+1),'/',len4,len(mat_df),'suitible comb.')
        # Renaming columns
        print('total combinations within distance',len(output))

        output.columns = ['Euclidean','Grid_No','Park_entry_No','grid_lon','grid_lat','Grid_coords_centroid','Grid_m_centroid',
                      'Park_No','Parkroad_osmid','Park_geom','Parkroad_coords_centroid','Parkroad_m_centroid','park_lon',
                      'park_lat','parkshare_walked','park_area','walk_area_m2']

        output = output[['Euclidean','Grid_No','Park_entry_No','Grid_coords_centroid','Grid_m_centroid','walk_area_m2',
                     'Park_No','Parkroad_osmid','Park_geom','Parkroad_coords_centroid','Parkroad_m_centroid','park_area']]

        # Reinstate geographic elements
        output = gpd.GeoDataFrame(output, geometry = 'Grid_coords_centroid', crs = 4326)
        output['Grid_m_centroid'] = gpd.GeoSeries(output['Grid_m_centroid'], crs = 3043)
        output['Parkroad_coords_centroid'] = gpd.GeoSeries(output['Parkroad_coords_centroid'], crs = 4326)
        output['Parkroad_m_centroid'] = gpd.GeoSeries(output['Parkroad_m_centroid'], crs = 3043)

        # Get the nearest entrance point for the grid centroids
        output = gridroad_entry(output, road_nodes[l])

        print('100 % gridentry done', round((time.time() - start_time) / 60,2),' mns')
        RoadComb.append(output)
    return (RoadComb)

In [27]:
def gridroad_entry (suitible_comb, road_nodes):    
    start_time = time.time()
    mat5 = list()
    for i in range(len(suitible_comb)):
        try:
            nearest = int(road_nodes['geometry'].sindex.nearest(suitible_comb['Grid_coords_centroid'].iloc[i])[1])
            mat5.append(road_nodes['osmid_var'].iloc[nearest])
        except: 
            # sometimes two nodes are the exact same distance, then the first in the list is taken.
            nearest = int(road_nodes['geometry'].sindex.nearest(suitible_comb['Grid_coords_centroid'].iloc[i])[1][0])
            mat5.append(road_nodes['osmid_var'].iloc[nearest])
        if i % 250000 == 0: print(round(i/len(suitible_comb)*100,1),'% gridentry done', round((time.time() - start_time) / 60,2),' mns')
    # format resulting dataframe
    suitible_comb['grid_osm'] = mat5
    suitible_comb = pd.merge(suitible_comb, road_nodes['geometry'], left_on = 'grid_osm', right_index = True)
    suitible_comb['geometry_m'] = gpd.GeoSeries(suitible_comb['geometry'], crs = 4326).to_crs(3043)
    suitible_comb = suitible_comb.reset_index()
    return(suitible_comb)

In [28]:
# Check grids in or out of UGS
def grids_in_UGS (suitible_comb, UGS, pop_grid): 
    start_time = time.time()
    RoadInOut = list()
    for i in range(len(suitible_comb)):
        UGS_geoms = UGS[i]['geometry'].to_crs(4326)
        grid = pop_grid[i]['centroid']
        lst = list()
        print('Check grids within UGS')
        for l in enumerate(UGS_geoms):
            lst.append(grid.intersection(l[1]).is_empty == False)
            if l[0] % 100 == 0: print(l[0], round((time.time() - start_time) / 60,2),' mns')

        dfGrUGS = pd.DataFrame(pd.DataFrame(np.array(lst)).unstack())
        dfGrUGS.columns = ['in_out_UGS']
        merged = pd.merge(suitible_comb[i], dfGrUGS, left_on = ['Grid_No','Park_No'], right_index = True, how = 'left')
        RoadInOut.append(merged)
    return(RoadInOut)    

In [29]:
# Block 7 calculate route networks of all grid-parkentry combinations within euclidean threshold distance
def route_finding (graphs, combinations, road_nodes, road_edges, cities, block_size = 250000, nn_iter = 10):

    warnings.filterwarnings("ignore")
    start_time = time.time()

    Routes = list()
    Routes_detail = list()
    for j in range(len(cities)):
        Graph = graphs[j]
        suit_raw = combinations[j] # iloc to test the iteration speed.
        nodes = road_nodes[j]

        In_UGS = suit_raw[suit_raw['in_out_UGS'] == True] # Check if a grid centroid is in an UGS
        suitible = suit_raw[suit_raw['in_out_UGS'] == False].reset_index(drop = True) # recreate a subsequential index
                                                                                      # for the other grids outside UGS
        block = block_size # Execute with chunks for performance improvement.

        Route_parts = pd.DataFrame()
        Route_dparts = pd.DataFrame()
        len2 = int(np.ceil(len(suitible)/block))
        # Divide in chunks of block for computational load
        for k in range(len2):    
            suitible_chunk = suitible.iloc[k*block:k*block+block] # Select chunk

            parknode = list(suitible_chunk['Parkroad_osmid'])
            gridnode = list(suitible_chunk['grid_osm'])

            s_mat = list([]) # origin (normally grid) osmid
            s_mat1 = list([]) # destination (normally UGS) osmid
            s_mat2 = list([]) # route id
            s_mat3 = list([]) # step id
            s_mat4 = list([]) # way calculated
            s_mat5 = list([]) # way calculated id
            mat_nn = [] # found nearest nodes by block
            len1 = len(suitible_chunk)

            print(cities[j].rsplit(',')[0], k+1,'/',len2,'range',k*block,'-',k*block+np.where(k*block+block >= len1,len1,block))
            for i in range(len(suitible_chunk)):
                try: 
                    # from grid to UGS.
                    shortest = nx.shortest_path(Graph, gridnode[i], parknode[i], 'travel_dist', method = 'dijkstra')
                    s_mat.append(shortest)
                    shortest_to = list(shortest[1:len(shortest)])
                    shortest_to.append(-1)
                    s_mat1.append(shortest_to)
                    s_mat2.append(list(np.repeat(i+block*k, len(shortest))))
                    s_mat3.append(list(np.arange(0, len(shortest))))
                    s_mat4.append('normal way')
                    s_mat5.append(1)
                except:
                    try:
                        # Check the reverse
                        shortest = nx.shortest_path(Graph, parknode[i], gridnode[i], 'travel_dist', method = 'dijkstra')
                        s_mat.append(shortest)
                        shortest_to = list(shortest[1:len(shortest)])
                        shortest_to.append(-1)
                        s_mat1.append(shortest_to)
                        s_mat2.append(list(np.repeat(i+block*k, len(shortest))))
                        s_mat3.append(list(np.arange(0, len(shortest))))
                        s_mat4.append('reverse way')
                        s_mat5.append(0)
                    except:
                        # Otherwise find nearest nodes (grid and UGS) and try to find routes between them
                        nn_route_finding(Graph, suitible_chunk, nodes, s_mat, s_mat1, s_mat2, s_mat3,
                                             s_mat4, s_mat5, mat_nn, i, block, k, nn_iter)
                        
                if i % 10000 == 0: print(round((i+block*k)/len(suitible)*100,2),'% done',
                                         round((time.time() - start_time) / 60,2),'mns')
            print('for', len(mat_nn),'routes nearest nodes found')

            print(round((i+block*k)/len(suitible)*100,2),'% pathfinding done', round((time.time() - start_time) / 60,2),'mns')

            # Formats route information by route and step (detailed)
            routes = route_formatting(s_mat, s_mat1, s_mat2, s_mat3, road_edges[j]) # Formats lists to routes detail.
            print('formatting done', round((time.time() - start_time) / 60,2), 'mns')
            
            # Summarizes information by route
            routes2 = route_summarization(routes, suitible_chunk, road_nodes[j], s_mat4, s_mat5) # formats routes to summary
            print('dissolving done', round((time.time() - start_time) / 60,2), 'mns')
            
            Route_parts = pd.concat([Route_parts, routes2])
            Route_dparts = pd.concat([Route_dparts, routes])

        # Format grids in UGS to enable smooth df concat
        In_UGS = In_UGS.set_geometry(In_UGS['Grid_coords_centroid'])
        In_UGS = In_UGS[['geometry','Grid_No','grid_osm','Park_No','Park_entry_No','Parkroad_osmid',
                                   'Grid_m_centroid','walk_area_m2',
                                   'Euclidean','geometry_m']]

        In_UGS['realG_osmid'] = suit_raw['Parkroad_osmid']
        In_UGS['realP_osmid'] = suit_raw['grid_osm']
        In_UGS['way_calc'] = 'grid in UGS'

        Route_parts = pd.concat([Route_parts,In_UGS])
        Route_parts = Route_parts.reset_index(drop = True)

        Route_parts['gridpark_no'] = Route_parts['Grid_No'].astype(str) +'-'+ Route_parts['Park_No'].astype(str)

        # All fill value 0 because no routes are calculated for grid centroids in UGSs
        to_fill = ['way-id','route_cost','steps','real_G-entry','Tcost']                                   
        Route_parts[to_fill] = Route_parts[to_fill].fillna(0)  
            
        Routes.append(Route_parts)
        Routes_detail.append(Route_dparts)
    return(Routes)

In [30]:
def nn_route_finding(graph, suitible_chunk, nodes, mat_from, mat_to, mat_route, mat_step,
                                             mat_way, mat_wbin, mat_nn, i, block, k, nn_iter):
                        
    # Order in route for nearest node:
    # 1. gridnode to nearest to the original failed parknode
    # 2. The reverse of 1.
    # 3. nearest gridnode to the failed one and route to park
    # 4. The reverse of 3.
                        
    gridosm = suitible_chunk['grid_osm'] # grid osmid
    UGSosm = suitible_chunk['Parkroad_osmid'] # UGS osmid
    nodeosm = nodes['osmid_var'] # road node osmid
    nodegeom = nodes['geometry'] # road node geometry
                        
    len3 = 0
    alt_route = list([])
    while len3 < nn_iter and len(alt_route) < 1: # If a route is found (alt_route == 1) or until max iterations

        len3 = len3 +1
                            
        nn = nn_finding(gridosm, UGSosm, nodeosm, nodegeom, nodes, i, len3) # finds nearest node.

        nn_routing (graph, nn['currUGS'], nn['nearUGS'], nn['currgrid'], nn['neargrid'], 
                                        mat_way, mat_wbin, len3, alt_route) # executes route finding in try order.
    if len(alt_route) == 0: 
        alt = alt_route 
    else: 
        alt = alt_route[0]
    len4 = len(alt)
    if len4 > 0: # If a route is found
        mat_nn.append(i+block*k)
        mat_from.append(alt)
        shortest_to = list(alt[1:len(alt)])
        shortest_to.append(-1)
        mat_to.append(shortest_to)
        mat_route.append(list(np.repeat(i+block*k,len4)))
        mat_step.append(list(np.arange(0, len4)))
    else: # If a route is not found
        mat_from.append(-1)
        mat_to.append(-1)
        mat_route.append(i+block*k)
        mat_step.append(-1)
        mat_way.append('no way')
        mat_wbin.append(2)
        print('index',i+block*k,'No route')

In [31]:
def nn_finding (gridosm, UGSosm, nodeosm, nodegeom, nodes, i, nn_i): 
    # Grid nearest
    g_geom = nodegeom[nodeosm == int(gridosm[i:i+1])] # Get geom of current node UGS
    g_nearest = pd.DataFrame((abs(float(g_geom.x) - nodegeom.x)**2 # Check distance UGS
    +abs(float(g_geom.y) - nodegeom.y)**2)**(1/2)
                            ).join(nodeosm).sort_values(0) # sort by distance ascending UGS

    g_grid = g_nearest.iloc[nn_i,1] # get the nearest node according to the nn_iter UGS entry
    g_park = list(UGSosm)[i] # current node
        
    p_geom = nodegeom[nodeosm == int(UGSosm[i:i+1])] # get the geom of the current node grid
    p_nearest = pd.DataFrame((abs(float(p_geom.x) - nodegeom.x)**2 # Check distance grid
    +abs(float(p_geom.y) - nodegeom.y)**2)**(1/2)
                            ).join(nodeosm).sort_values(0) # sort by distance ascending grid

    p_grid = list(gridosm)[i] # current node
    p_park = p_nearest.iloc[nn_i,1] # get the nearest node to the nn_iter grid
    return({'currUGS':p_grid, 'nearUGS':p_park,'currgrid':g_park, 'neargrid':g_grid})

In [32]:
# Improve: 2-to-2 instead of 1-to-all.

def nn_routing (graph, curr_UGS, near_UGS, curr_grid, near_grid, mat_way, mat_wbin, nn_i, found_route):
    try:
        found_route.append(nx.shortest_path(graph, curr_UGS, near_UGS, 
                                          'travel_dist', method = 'dijkstra'))
        mat_way.append(str(nn_i)+'grid > n-park') # grid to nearest unseen UGS node
        mat_wbin.append(1)
    except:
        try:
            found_route.append(nx.shortest_path(graph, near_UGS, curr_UGS, 
                                              'travel_dist', method = 'dijkstra'))
            mat_way.append(str(nn_i)+'n-park > grid') # nearest unseen UGS node to grid
            mat_wbin.append(0)
        except:
            try:
                found_route.append(nx.shortest_path(graph, curr_grid, near_grid, 
                                                  'travel_dist', method = 'dijkstra'))
                mat_way.append(str(nn_i)+'n-grid > park') # nearest grid node to UGS
                mat_wbin.append(1)
            except:
                try:
                    found_route.append(nx.shortest_path(graph, near_grid, curr_grid, 
                                                      'travel_dist', method = 'dijkstra'))
                    mat_way.append(str(nn_i)+'park > n-grid') # UGS to nearest grid node
                    mat_wbin.append(0)
                except:
                    try:
                        found_route.append(nx.shortest_path(graph, near_grid, near_UGS, 
                                                      'travel_dist', method = 'dijkstra'))
                        mat_way.append(str(nn_i)+'park > n-grid') # UGS to nearest grid node
                        mat_wbin.append(0)
                    except:
                        try:
                            found_route.append(nx.shortest_path(graph, near_UGS, near_grid, 
                                                      'travel_dist', method = 'dijkstra'))
                            mat_way.append(str(nn_i)+'park > n-grid') # UGS to nearest grid node
                            mat_wbin.append(1)
                        except:
                            pass

In [33]:
def route_formatting(mat_from, mat_to, mat_route, mat_step, road_edges):
    # Unpack lists
    s_mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, mat_from) for i in b]
    s_mat_u1 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, mat_to) for i in b]
    s_mat_u2 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, mat_route) for i in b]
    s_mat_u3 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, mat_step) for i in b]

    # Format df
    routes = pd.DataFrame([s_mat_u,s_mat_u1,s_mat_u2,s_mat_u3]).transpose()
    routes.columns = ['from','to','route','step']
    mat_key = list([])
    for n in range(len(routes)): # get key of origin and destination
        mat_key.append(str(int(s_mat_u[n])) + '-' + str(int(s_mat_u1[n])))
    routes['key'] = mat_key
    routes = routes.set_index('key')

    # Add route information
    routes = routes.join(road_edges, how = 'left') # to add road node information
    routes = gpd.GeoDataFrame(routes, geometry = 'geometry', crs = 4326)
    routes = routes.sort_values(by = ['route','step'])
    return(routes)

In [34]:
def route_summarization(routes, suitible_comb, road_nodes, mat_way, mat_wbin):
    # dissolve route
    routes2 = routes[['route','geometry']].dissolve('route')

    # get used grid- and parkosm. Differs at NN-route.
    route_reset = routes.reset_index()
    origin = route_reset['from'].iloc[list(route_reset.groupby('route')['step'].idxmin()),]
    origin = origin.reset_index().iloc[:,-1]
    dest = route_reset['from'].iloc[list(route_reset.groupby('route')['step'].idxmax()),]
    dest = dest.reset_index().iloc[:,-1]

    # grid > park = 1, park > grid = 0, no way = 2, detailed way in way_calc.
    routes2['way-id'] = mat_wbin
    routes2['realG_osmid'] = np.where(routes2['way-id'] == 1, origin, dest)
    routes2['realP_osmid'] = np.where(routes2['way-id'] == 1, dest, origin)
    routes2['way_calc'] = mat_way

    # get route cost, steps, additional information.
    routes2['route_cost'] = routes.groupby('route')['length'].sum()
    routes2['steps'] = routes.groupby('route')['step'].max()
    routes2['index'] = suitible_comb.index
    routes2 = routes2.set_index(['index'])
    routes2.index = routes2.index.astype(int)
    routes2 = pd.merge(routes2, suitible_comb[['Grid_No','grid_osm','Park_No','Park_entry_No','Parkroad_osmid',
                                          'Grid_m_centroid','walk_area_m2','Euclidean']],
                                            left_index = True, right_index = True)
    routes2 = pd.merge(routes2, road_nodes['geometry_m'], how = 'left', left_on = 'realG_osmid', right_index = True)
    # calculate distance of used road-entry for grid-centroid.
    routes2['real_G-entry'] = round(gpd.GeoSeries(routes2['Grid_m_centroid'], crs = 3043).distance(routes2['geometry_m']),3)
                                    
    # Calculcate total route cost for the four gravity variants
    routes2['Tcost'] = routes2['route_cost'] + routes2['real_G-entry']
    return(routes2)

In [35]:
def min_gridUGS_comb (routes, grids, UGS):
    gp_nearest = []
    for i in range(len(routes)):
        gp_nn = routes[i][routes[i]['Tcost'] <= max(thresholds)]
        gp_nn = pd.merge(gp_nn, grids[i]['population'], left_on='Grid_No', right_index = True)
        gp_nn = pd.merge(gp_nn, UGS[i]['park_area'], left_on = 'Park_No', right_index = True)
        gp_nn = gp_nn.reset_index()

        gp_nn = gp_nn.iloc[gp_nn.groupby('gridpark_no')['Tcost'].idxmin()]
        gp_nn.index.name = 'idx'
        gp_nn = gp_nn.sort_values('idx')
        gp_nn = gp_nn.reset_index()
        gp_nearest.append(gp_nn)
    gp_nearest[0].sort_values('Grid_No')
    return(gp_nearest)

In [36]:
def E2SCFA_scores(min_gridUGS_comb, grids, thresholds, cities, save_path = 'C:/Dumps/GEE-WP Scores/E2SFCA/', grid_size = 100):
    pd.options.display.float_format = '{:20,.2f}'.format
    E2SFCA_cities = []
    E2SFCA_summary = pd.DataFrame()
    for i in range(len(cities)):
        E2SFCA_score = grids[i][['population','geometry']]
        for j in range(len(thresholds)):
            subset = min_gridUGS_comb[i][min_gridUGS_comb[i]['Tcost'] <= thresholds[j]]

            # use gussian distribution: let v= 923325, then the weight for 800m is 0.5
            v = -thresholds[j]**2/np.log(0.5)

            # add a column of weight: apply the decay function on distance
            subset['weight'] = np.exp(-(subset['Tcost']**2/v)).astype(float)
            subset['pop_weight'] = subset['weight'] * subset['population']

            # get the sum of weighted population each green space has to serve.
            s_w_p = pd.DataFrame(subset.groupby('Park_No').sum('pop_weight')['pop_weight'])

            # delete other columns, because they are useless after groupby
            s_w_p = s_w_p.rename({'pop_weight':'pop_weight_sum'},axis = 1)
            middle = pd.merge(subset,s_w_p, how = 'left', on = 'Park_No' )

            # calculate the supply-demand ratio for each green space
            middle['green_supply'] = middle['park_area']/middle['pop_weight_sum']

            # caculate the accessbility score for each green space that each population grid cell could reach
            middle['Sc-access'] = middle['weight'] * middle['green_supply']
            # add the scores for each population grid cell
            pop_score_df = pd.DataFrame(middle.groupby('Grid_No').sum('Sc-access')['Sc-access'])

            # calculate the mean distance of all the green space each population grid cell could reach
            mean_dist = middle.groupby('Grid_No').mean('Tcost')['Tcost']
            pop_score_df['M-dist'] = mean_dist

            # calculate the mean area of all the green space each population grid cell could reach
            mean_area = middle.groupby('Grid_No').mean('park_area')['park_area']
            pop_score_df['M-area'] = mean_area

            # calculate the mean supply_demand ratio of all the green space each population grid cell could reach
            mean_supply = middle.groupby('Grid_No').mean('green_supply')['green_supply']
            pop_score_df['M-supply'] = mean_supply

            pop_score = pop_score_df

            pop_score_df = pop_score_df.join(grids[i]['population'], how = 'right')
            pop_score_df['Sc-norm'] = pop_score_df['Sc-access'] / pop_score_df['population']

            pop_score_df = pop_score_df.loc[:, pop_score_df.columns != 'population']
            pop_score_df = pop_score_df.add_suffix(' '+str(thresholds[j]))
            E2SFCA_score = E2SFCA_score.join(pop_score_df, how = 'left')

            print(thresholds[j], cities[i])

        E2SFCA_score = E2SFCA_score.fillna(0)
        
        if not os.path.exists(save_path+str(grid_size)+'m grids'+'/grid_geoms/'):
            os.makedirs(save_path+str(grid_size)+'m grids'+'/grid_geoms/')
        
        E2SFCA_score.to_file(save_path+str(grid_size)+'m grids'+'/grid_geoms/'+cities[i]+'.gpkg') # Detailed scores
        pop_sum = pd.Series(E2SFCA_score['population'].sum()).astype(int)
        pop_sum.index = ['population']
        mean_metrics = E2SFCA_score.loc[:, E2SFCA_score.columns != 'population'].mean()
        E2SFCA_sum = pd.concat([pop_sum, mean_metrics])
        E2SFCA_summary = pd.concat([E2SFCA_summary, E2SFCA_sum], axis = 1) # summarized results
        E2SFCA_cities.append(E2SFCA_score)
        
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        
        E2SFCA_score.loc[:, E2SFCA_score.columns != 'geometry'].to_csv(save_path+cities[i]+'.csv')
    E2SFCA_summary.columns = cities
    
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    E2SFCA_summary.to_csv(save_path+str(grid_size)+'m grids'+'all_cities.csv')
    E2SFCA_summary
    return({'score summary':E2SFCA_summary,'score detail':E2SFCA_cities})